In [1]:
import pyspark
import os
import sys
from pyspark.sql import SparkSession

In [2]:
os.environ['PYSPARK_PYTHON']=sys.executable
os.environ['PYSPARK_DRIVER_PYTHON']=sys.executable

spark=SparkSession.builder.getOrCreate()

In [8]:
path=r"C:\Users\LENOVO\Desktop\SUB 6TH SEM\BDA\LAB\LAB3\record+linkage+comparison+patterns\donation\block_1\block_1.csv"
data=spark.read.csv(path,inferSchema=True,header=True)

In [9]:
data.show()

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|           ?|         1.0|           ?|      1|     1|     1|     1|      0|    true|
|39086|47614|                1|           ?|         1.0|           ?|      1|     1|     1|     1|      1|    true|
|70031|70237|                1|           ?|         1.0|           ?|      1|     1|     1|     1|      1|    true|
|84795|97439|                1|           ?|         1.0|           ?|      1|     1|     1|     1|      1|    true|
|36950|42116|                1|           ?|         1.0|           1|      1|     1|     1|     1|      1|    true|
|42413|48491|                1|           ?|         1.0|       

In [12]:
from pyspark.sql.functions import col
data.select('is_match').groupBy('is_match').count().orderBy(col('count').desc()).show()

+--------+------+
|is_match| count|
+--------+------+
|   false|572820|
|    true|  2093|
+--------+------+



In [16]:
data.createOrReplaceTempView('linkage')
spark.sql("""select is_match, count(*) cnt from linkage group by is_match order by cnt desc""").show()

+--------+------+
|is_match|   cnt|
+--------+------+
|   false|572820|
|    true|  2093|
+--------+------+



In [21]:
#Fast Summary Statistics for DataFrames

summary=data.describe()
summary.printSchema()
summary.select("summary","cmp_fname_c1","cmp_fname_c2").show()

root
 |-- summary: string (nullable = true)
 |-- id_1: string (nullable = true)
 |-- id_2: string (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: string (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: string (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: string (nullable = true)
 |-- cmp_plz: string (nullable = true)

+-------+------------------+------------------+
|summary|      cmp_fname_c1|      cmp_fname_c2|
+-------+------------------+------------------+
|  count|            574913|            574913|
|   mean|0.7127592938253411|0.8977586763518969|
| stddev|0.3889286452463531|0.2742577520430532|
|    min|                 0|                 0|
|    max|                 ?|                 ?|
+-------+------------------+------------------+



In [102]:
path=r"C:\Users\LENOVO\Desktop\SUB 6TH SEM\BDA\LAB\LAB3\record+linkage+comparison+patterns\donation\block_1\block_1.csv"
data=spark.read.csv(path,inferSchema=True,header=True)
matches=data.filter(col('is_match')=='true')
match_summary=matches.describe()
matches.show(5) 
misses=data.filter(col('is_match')=='false')
miss_summary=misses.describe()
miss_summary.show()

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|           ?|         1.0|           ?|      1|     1|     1|     1|      0|    true|
|39086|47614|                1|           ?|         1.0|           ?|      1|     1|     1|     1|      1|    true|
|70031|70237|                1|           ?|         1.0|           ?|      1|     1|     1|     1|      1|    true|
|84795|97439|                1|           ?|         1.0|           ?|      1|     1|     1|     1|      1|    true|
|36950|42116|                1|           ?|         1.0|           1|      1|     1|     1|     1|      1|    true|
+-----+-----+-----------------+------------+------------+-------

In [59]:
#Pivoting and Reshaping DataFrames
from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DoubleType

summary_p=summary.toPandas()
summary_p=summary_p.set_index('summary').transpose().reset_index()
summary_p=summary_p.rename(columns={'index':'field'})
summary_p=summary_p.rename_axis(None,axis=1)
summary_t=spark.createDataFrame(summary_p)

for c in summary_t.columns:
    if c=='field':
        continue
    else:
        summary_t=summary_t.withColumn(c,summary_t[c].cast(DoubleType()))

summary_t.show()

+------------+--------+--------------------+-------------------+---+--------+
|       field|   count|                mean|             stddev|min|     max|
+------------+--------+--------------------+-------------------+---+--------+
|        id_1|574913.0|  33271.962171667714| 23622.669425933756|1.0| 99894.0|
|        id_2|574913.0|    66564.6636865056|  23642.00230967228|6.0|100000.0|
|cmp_fname_c1|574913.0|  0.7127592938253411| 0.3889286452463531|0.0|    NULL|
|cmp_fname_c2|574913.0|  0.8977586763518969| 0.2742577520430532|0.0|    NULL|
|cmp_lname_c1|574913.0|  0.3155724578100624| 0.3342494687554245|0.0|     1.0|
|cmp_lname_c2|574913.0|  0.3269155414552904| 0.3783092020540671|0.0|    NULL|
|     cmp_sex|574913.0|  0.9550923357099248|0.20710152240504442|0.0|     1.0|
|      cmp_bd|574913.0| 0.22475563232907309| 0.4174216587235557|0.0|    NULL|
|      cmp_bm|574913.0|  0.4886361857246487| 0.4998712818281637|0.0|    NULL|
|      cmp_by|574913.0| 0.22266639529199742| 0.4160365041645591|

In [103]:
def pivot_summary(data):
    data=data.toPandas()
    data=data.set_index('summary').transpose().reset_index()
    data=data.rename(columns={'index':'field'})
    data=data.rename_axis(None,axis=1)
    data=spark.createDataFrame(data)
    for c in data.columns:
        if c=='field':
            continue
        else:
            data.withColumn(c,data[c].cast(DoubleType()))
    return data 

In [104]:
miss_summary=pivot_summary(miss_summary)
match_summary=pivot_summary(match_summary)
match_summary.show()

+------------+-----+------------------+--------------------+-----------------+-----+
|       field|count|              mean|              stddev|              min|  max|
+------------+-----+------------------+--------------------+-----------------+-----+
|        id_1| 2093| 34440.86956521739|  21743.893698458884|                5|98459|
|        id_2| 2093| 50889.32345914955|    24382.2927756227|                6|99897|
|cmp_fname_c1| 2093|0.9970329792424486| 0.03979189523588238|                0|    ?|
|cmp_fname_c2| 2093|0.9955357142857143| 0.05050762722761048|0.428571428571429|    ?|
|cmp_lname_c1| 2093| 0.997560095051734|0.039218983113320234|              0.0|  1.0|
|cmp_lname_c2| 2093| 0.960167714884696| 0.17815710016004582|                0|    ?|
|     cmp_sex| 2093|0.9894887720974678| 0.10200839997032067|                0|    1|
|      cmp_bd| 2093|0.9952221691352127| 0.06897301025106671|                0|    1|
|      cmp_bm| 2093|0.9976110845676063| 0.04882978308841625|     

In [107]:
match_summary.createOrReplaceTempView('match_view')
miss_summary.createOrReplaceTempView('miss_view')

In [110]:
spark.sql("""
    SELECT 
        a.field, 
        a.count + b.count AS total, 
        a.mean - b.mean AS delta 
    FROM 
        match_view AS a 
    INNER JOIN 
        miss_view AS b 
    ON 
        a.field = b.field 
    ORDER BY 
        delta DESC, 
        total DESC
""").show()


+------------+--------+-------------------+
|       field|   total|              delta|
+------------+--------+-------------------+
|        id_1|574913.0| 1173.1784091823356|
|     cmp_plz|574913.0| 0.9524975516429005|
|cmp_lname_c2|574913.0| 0.8136949970410103|
|      cmp_by|574913.0| 0.7763379425859384|
|      cmp_bd|574913.0| 0.7732820129086737|
|cmp_lname_c1|574913.0| 0.6844795197261291|
|      cmp_bm|574913.0|  0.510834819548174|
|cmp_fname_c1|574913.0|0.28531156828518667|
|cmp_fname_c2|574913.0|0.09900440489032714|
|     cmp_sex|574913.0|0.03452211590529575|
|        id_2|574913.0|-15732.615614206828|
+------------+--------+-------------------+



In [111]:
data.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: string (nullable = true)
 |-- cmp_plz: string (nullable = true)
 |-- is_match: boolean (nullable = true)



In [114]:
good_features=['cmp_lname_c1','cmp_plz','cmp_bd','cmp_bm']
sum_expression='+'.join(good_features)
sum_expression

'cmp_lname_c1+cmp_plz+cmp_bd+cmp_bm'

In [120]:
from pyspark.sql.functions import expr
score=data.fillna(0,good_features).withColumn('score',expr(sum_expression)).select('is_match','score')
score.show()

+--------+-----+
|is_match|score|
+--------+-----+
|    true|  3.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  3.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  3.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
|    true|  4.0|
+--------+-----+
only showing top 20 rows



In [133]:
def crossTabs(scored: DataFrame, t: DoubleType) -> DataFrame:
    return scored.selectExpr(f"score>={t} as above","is_match").groupBy("above").pivot("is_match",("true","false")).count()

In [135]:
cm1=crossTabs(score,4.0)
cm1.show()

+-----+----+------+
|above|true| false|
+-----+----+------+
| NULL|   4|  1353|
| true|1967|  NULL|
|false| 122|571467|
+-----+----+------+



In [139]:
tp=cm1.filter("above==true").select("true").collect()[0].true
tn=cm1.filter("above==true").select("false").collect()[0].false
fn=cm1.filter("above==false").select('false').collect()[0].false
fp=cm1.filter("above==false").select('true').collect()[0].true

In [140]:
acc=tp/(tp+fp)
print(acc)

0.9415988511249401


In [141]:
recall=tp/(tp+fn)
print(recall)

0.0034302116721366366
